In [1]:
import pandas as pd, dataprep as dp 

In [2]:
data = pd.read_csv("/Users/florian/Documents/github/study/IoT/IoT/main/output.csv")

a0 = ["hka-aqm-a017", "hka-aqm-a014"]
a1 = ["hka-aqm-a101", "hka-aqm-a102", "hka-aqm-a103", "hka-aqm-a106", "hka-aqm-a107", "hka-aqm-a108", "hka-aqm-a111", "hka-aqm-a112"]
data = data[data["device_id"].isin(a0 + a1)]  
data["device_id"] = data["device_id"].str.replace("hka-aqm-", "")

In [3]:
aggregation = "h"

df = dp.group_data(data, aggregation)
df["hour"] = df["date_time"].dt.hour
df["day"] = df["date_time"].dt.date
df['week'] = df['date_time'].dt.to_period('W').apply(lambda r: r.start_time)
df['month'] = df['date_time'].dt.to_period('M').apply(lambda r: r.start_time)
df["year"] = df["date_time"].dt.year

In [15]:
unique_years = df["year"].unique()
year_labels = [pd.to_datetime(str(year), format='%Y').strftime('%Y') for year in unique_years]
year_labels

['2022', '2023']

In [32]:
import numpy as np
input_agg = "min"

def get_unique_values_and_labels(df, column, format):
    unique_values, indices = np.unique(df[column], return_inverse=True)
    if column == "min":
        labels = [f"{value}:00" for value in unique_values]
    elif column == "year":
        labels = [pd.to_datetime(str(value), format='%Y').strftime('%Y') for value in unique_values]
    elif column == "week":
        labels = [f"{value.strftime('%d.%m.%Y')} - {(value + pd.DateOffset(days=6)).strftime('%d.%m.%Y')}" for value in unique_values]
    else:
        labels = [value.strftime(format) for value in unique_values]
    return unique_values[indices], labels

def get_filtered_data(df, column, selected_label, unique_values, labels):
    selected_value = unique_values[labels.index(selected_label)]
    return df[df[column] == selected_value]

df = dp.group_data(data, input_agg)
df["min"] = df["date_time"].dt.minute
df["hour"] = df["date_time"].dt.hour
df["day"] = df["date_time"].dt.date
df['week'] = df['date_time'].dt.to_period('W').apply(lambda r: r.start_time)
df['month'] = df['date_time'].dt.to_period('M').apply(lambda r: r.start_time)

aggregation_dict = {
    'min': ('hour', 'Select minute', '%Y-%m-%d %H:%M'),
    'h': ('min', 'Select hour', '%Y-%m-%d %H'),
    'd': ('hour', 'Select day', '%Y-%m-%d'),
    'w': ('day', 'Select week', '%d.%m.%Y'),
    'm': ('week', 'Select month', '%m/%Y'),
    'y': ('month', 'Select year', '%Y')
}

df["day"]

0         2022-10-10
1         2022-10-10
2         2022-10-10
3         2022-10-10
4         2022-10-10
             ...    
199097    2023-10-01
199098    2023-10-01
199099    2023-10-01
199100    2023-10-01
199101    2023-10-01
Name: day, Length: 199102, dtype: object

In [26]:
input_agg = "h"

aggregation_dict = {
    'h': ('min', 'Select hour', '%Y-%m-%d %H'),
    'd': ('hour', 'Select day', '%Y-%m-%d'),
    'w': ('day', 'Select week', '%d.%m.%Y'),
    'm': ('week', 'Select month', '%m/%Y'),
    'y': ('month', 'Select year', '%Y')
}

aggregation_dict[input_agg][0]

'min'